In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add, coroutine_create
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

def DefinePISO(n):
    @si.coroutine(inputs={"SI": si.Bit, "PI": si.Bits(n), "LOAD": si.Bit})
    def PISO():
        values = bits(0, n)
        O = values[-1]
        while True:
            SI, PI, LOAD = yield O
            O = values[-1]
            if LOAD:
                values = PI
            else:
                # values = [SI] + values[:-1]
                for i in range(n - 1, 0, -1):
                    values[i] = values[i - 1]
                values[0] = SI
    return PISO



@si.coroutine(inputs={"valid": si.Bit, "message": si.Bits(8)})
def UART_TX():
    piso = coroutine_create(DefinePISO(8))
    valid, message = yield
    while True:
        O = piso.send((1, message, valid))
        if valid:
            O = False
            valid, message = yield O
            for j in range(9):
                O = piso.send((1, message, 0))
                valid, message = yield O
        else:
            valid, message = yield O
        
uart = si.compile(UART_TX(), file_name="build/silica_uart.py")
with open("build/silica_uart.py", "r") as magma_file:
    print(magma_file.read())

import mantle lattice ice40
import mantle lattice mantle40
from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width):
    if "one-hot" == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
                        

In [6]:
from magma.bitutils import int2seq, seq2int
messages = [0xDE, 0xAD, 0xBE, 0xEF]
uart_tx = UART_TX()
  
@si.coroutine
def inputs_generator(messages):
    while True:
        for message in messages:
            valid = True
            message = int2seq(message, 8)
            yield valid, message
            for j in range(9):
                valid = False
                message = int2seq(0, 8)
                yield valid, message
                
inputs = inputs_generator(messages)
for i in range(4):
    message = []
    for j in range(10):
        uart_tx.send((inputs.valid, inputs.message))
        next(inputs)
        message.insert(0, uart_tx.O)
    print(f"Got      : {seq2int(message[1:-1])}")
    print(f"Expected : {messages[i]}")
    assert message[0] == 1
    assert message[-1] == 0
    assert seq2int(message[1:-1]) == messages[i]


check(uart, UART_TX(), len(messages) * 10, inputs_generator(messages))
print("Passed!")

Got      : 222
Expected : 222
Got      : 173
Expected : 173
Got      : 190
Expected : 190
Got      : 239
Expected : 239
Passed!


In [3]:
!magma -o coreir -m coreir -t UART_TX build/silica_uart.py
!coreir -i build/silica_uart.json -o build/silica_uart.v
!yosys -p 'synth_ice40 -top UART_TX -blif build/silica_uart.blif' build/silica_uart.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_uart.txt build/silica_uart.blif
!icetime -tmd hx1k build/silica_uart.txt  | grep -B 2 "Total path delay"

In Run Generators
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!

Modified?: Yes
2.27. Printing statistics.

=== UART_TX ===

   Number of wires:               1160
   Number of wire bits:           4599
   Number of public wires:        1155
   Number of public wire bits:    4594
   Number of memories:               0
   Number of memory bits:            0
   Number o

In [7]:
import sys
from magma import *
from mantle import Counter, Decode, DFF, LUT3, LUT2, I0, I1
from mantle import DefinePISO as MantleDefinePISO

main = DefineCircuit("main4", "valid", In(Bit), "message", In(Bits(8)), "O", Out(Bit), "CLK", In(Clock))

count = Counter(4, has_reset=True)
done = Decode(10, 4)(count)

run = DFF()
run_n = LUT3([0,0,1,0, 1,0,1,0])
run_n(done, main.valid, run)
run(run_n)

reset = LUT2(I0&~I1)(done, run)
load = LUT2(I0&~I1)(main.valid,run)
wire(count.RESET, reset)

shift = MantleDefinePISO(9)()
shift(1, concat(bits(0, 1), main.message), load)
wire(shift.O, main.O)
EndDefine()

compile("build/magma_uart", main)

check(main, UART_TX(), len(messages) * 10, inputs_generator(messages))

compiling FullAdder
compiling Add4Cout
compiling Register4R
compiling Counter4R
compiling Mux2
compiling Register9
compiling PISO9
compiling main4


KeyError: QualifiedInstance(instance=inst1 = SB_LUT4(LUT_INIT=0x400), scope=<magma.scope.Scope object at 0x110ba9ac8>)

In [ ]:
!yosys -p 'synth_ice40 -top main -blif build/magma_uart.blif' build/magma_uart.v | grep -A 15 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/magma_uart.txt build/magma_uart.blif
!icetime -tmd hx1k build/magma_uart.txt | grep -B 2 "Total path delay"